In [1]:
import numpy as np
import os
import sys
import copy

import torch
torch.set_num_threads=4
from torch import nn
import scipy as sp

import importlib
import pickle as pk

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as anm

sys.path.insert(0, "../src/")
import data
import model
import train

#import nmrglue as ng
import scipy as sp
import scipy.interpolate as ip
import scipy.optimize as op
import scipy.signal as sg

import tqdm.auto as tqdm

np.random.seed(123)

In [2]:
exp_dir = f"../data/experimental_spectra/topspin/4096/"
exp_compounds = ["ampicillin", "aspala", "flutamide", "histidine", "thymol", "tyrosine", "mdma"]
exp_range = {"ampicillin": [1500, 2500],
             "aspala": [1500, 2500],
             "flutamide": [1500, 2500],
             "histidine": [1500, 2500],
             "thymol": [1500, 2500],
             "tyrosine": [1500, 2500],
             "mdma": [1600, 2400],
            }
align_regions = {"ampicillin": [12., 9.],
                 "aspala": [15., 11.],
                 "flutamide": [11., 9.],
                 "histidine": [18., 16.],
                 "thymol": [6.7, 5.7],
                 "tyrosine": [14., 11.],
                 "mdma": [4., 2.],
                }

align_ind = -1

fit_ranges = {"ampicillin": [[9.5, 12.]],
              "aspala": [[12., 15.], [-2., 1.5]],
              "flutamide": [[9.5, 12.]],
              "histidine": [[16., 20.], [11.5, 13.5], [4.5, 6.]],
              "thymol": [[8.5, 15.]],
              "tyrosine": [[12., 14.], [9.5, 11.5], [-2., 3.]],
              "mdma": [[1., 3.8]]}
bounds = [(None, None), (1.e-6, None), (0., 1.), (1e-6, None)]
bounds_v = [(None, None), (1.e-6, None), (1.e-6, None), (1e-6, None)]

ylabs_v = ["p", "wg", "wl", "h"]
ylabs_s = ["p", "w", "m", "h"]
ylabs_p = ["p", "w", "m", "h"]

w0s = np.linspace(0.1, 1, 10)
min_w_fit = 40000.

fig_dir = "../figures/GL_fitting/"
if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)

# Load experimental data

In [3]:
def inverse_mas_fit(wr, a, b):
    
    y = a + b / wr
    
    return y

In [4]:
def inverse_mas2_fit(wr, a, b, c):
    
    y = a + b / wr + c / (np.square(wr))
    
    return y

In [5]:
def load_topspin_spectrum(d):
    
    pd = f"{d}pdata/1/"
    
    fr = pd + "1r"
    fi = pd + "1i"

    with open(fr, "rb") as F:
        dr = np.fromfile(F, np.int32).astype(float)

    with open(fi, "rb") as F:
        di = np.fromfile(F, np.int32).astype(float)

    with open(f"{d}acqus", "r") as F:
        lines = F.read().split("\n")

    for l in lines:
        if l.startswith("##$MASR"):
            wr = int(l.split("=")[1].strip())
        if l.startswith("##$TD="):
            TD = int(l.split("=")[1].strip())
        if l.startswith("##$SW_h="):
            SW = float(l.split("=")[1].strip())

    with open(f"{pd}procs", "r") as F:
        lines = F.read().split("\n")

    for l in lines:
        if l.startswith("##$SI="):
            n_pts = int(l.split("=")[1].strip())

        if l.startswith("##$OFFSET="):
            offset = float(l.split("=")[1].strip())

        if l.startswith("##$SF="):
            SF = float(l.split("=")[1].strip())
            
    AQ = TD / (2 * SW)

    hz = offset * SF - np.arange(n_pts) / (2 * AQ * n_pts / TD)
    
    ppm = hz / SF

    return dr, di, wr, ppm, hz

In [6]:
def extract_exp_topspin(in_dir, compound):
    
    d0 = f"{in_dir}{compound}/"
    
    ws = []
    X = []
    
    for d in os.listdir(d0):
        if d.isnumeric():
            Xi, _, wr, ppm, hz = load_topspin_spectrum(f"{d0}{d}/")
            X.append(Xi)
            ws.append(wr)
    
    sorted_inds = np.argsort(ws)
    
    sorted_ws = np.array([ws[i] for i in sorted_inds])
    
    sorted_X = np.array([X[i] for i in sorted_inds])
    
    return ppm, sorted_ws, sorted_X

In [7]:
def get_maximum(ppm, X, r, method="direct"):
    
    r0 = min(r)
    r1 = max(r)
    
    if method == "direct":
        
        inds = np.where(np.logical_and(ppm > r0, ppm < r1))[0]
        
        i0 = np.argmax(X[inds])
        w0 = ppm[inds[i0]]
        
    elif method == "interp":
        
        inds = np.where(np.logical_and(ppm > r0, ppm < r1))[0]
        f = ip.interp1d(range(len(inds)), X[inds], kind="cubic")
        
        x = np.linspace(0, len(inds)-1, 1001)
        x_ppm = ppm[inds[0]] + x * (ppm[inds[1]] - ppm[inds[0]])
        y = f(x)
        w0 = x_ppm[np.argmax(y)]
        
    else:
        raise ValueError(f"Unknown method: {method}")
    
    return w0

In [8]:
def shift_spectrum(hz, Xr, Xi, dw):
    
    n = Xr.shape[0]
    t = np.arange(n) / np.abs(hz[1] - hz[0]) / n
    
    X = Xr + 1j * Xi
    
    T = np.fft.ifft(X)
    
    T *= np.exp(1j*dw*t * 2 * np.pi)
    X = np.fft.fft(T)
    Xr = np.real(X)
    Xi = np.imag(X)
    
    return Xr, Xi

In [9]:
def extract_and_shift_exp_topspin(in_dir, compound, align_region, align_ind=-1, method="interp"):
    
    d0 = f"{in_dir}{compound}/"
    
    ws = []
    X_real = []
    X_imag = []
    for d in os.listdir(d0):
        if d.isnumeric():
            Xr, Xi, wr, ppm, hz = load_topspin_spectrum(f"{d0}{d}/")
            X_real.append(Xr)
            X_imag.append(Xi)
            ws.append(wr)
    
    sorted_inds = np.argsort(ws)
    
    sorted_ws = np.array([ws[i] for i in sorted_inds])
    
    sorted_Xr = np.array([X_real[i] for i in sorted_inds])
    sorted_Xi = np.array([X_imag[i] for i in sorted_inds])
    
    # Extract target shift
    align_region_hz = [hz[np.argmin(np.abs(ppm - align_region[0]))], hz[np.argmin(np.abs(ppm - align_region[1]))]]
    w0 = get_maximum(hz, sorted_Xr[align_ind], align_region_hz, method=method)
    w0_ppm = get_maximum(ppm, sorted_Xr[align_ind], align_region, method=method)
    
    shifted_Xr = []
    shifted_Xi = []
    all_dw = []
    all_dw_ppm = []
    
    all_w = []
    for Xr, Xi in zip(sorted_Xr, sorted_Xi):
        # Get actual shift
        this_w = get_maximum(hz, Xr, align_region_hz, method=method)
        this_w_ppm = get_maximum(ppm, Xr, align_region, method=method)
        dw = this_w - w0
        all_dw.append(this_w- w0)
        all_dw_ppm.append(this_w_ppm- w0_ppm)
        
        # Shift spectrum
        Xr2, Xi2 = shift_spectrum(hz, Xr, Xi, dw)
        
        shifted_Xr.append(Xr2)
        shifted_Xi.append(Xi2)
    
    return ppm, sorted_ws, np.array(shifted_Xr), np.array(shifted_Xi), np.array(all_dw), np.array(all_dw_ppm), sorted_Xr, sorted_Xi

In [10]:
ppms = []
Xs = []
all_ws = []
for compound in exp_compounds:
    r = exp_range[compound]
    print(compound)
    ppm, ws, X, _, all_dw, all_dw_ppm, X0, _ = extract_and_shift_exp_topspin(exp_dir, compound, align_regions[compound], align_ind=align_ind)

    ppm = ppm[r[0]:r[1]]
    X = X0[:, r[0]:r[1]]

    X /= np.sum(X, axis=1)[:, np.newaxis]
    
    ppms.append(ppm)
    Xs.append(X)
    all_ws.append(ws)

ampicillin
aspala
flutamide
histidine
thymol
tyrosine
mdma


In [11]:
def voigt(x, p, wg, wl, h, eps=1e-12):
    
    G = np.exp(-4 * np.log(2) * np.square(x-p) / (wg ** 2))
    L = 1 / (1 + 4 * np.square(x-p) / (wl ** 2))
    
    V = sg.convolve(G, L, mode="same")
    
    if np.max(V) < eps:
        return V
    
    V /= np.max(V) / h
    
    return V

In [12]:
def gls(x, p, w, m, h):
    
    y = h * (1-m) * np.exp(-4 * np.log(2) * np.square(x-p) / (w ** 2))
    
    y += h * m / (1 + 4 * np.square(x-p) / (w ** 2))
    
    return y

In [13]:
def glp(x, p, w, m, h):
    
    y = h * np.exp(-4 * np.log(2) * (1-m) * np.square(x-p) / (w ** 2))
    
    y /= (1 + 4 * m * np.square(x-p) / (w ** 2))
    
    return y

In [14]:
def rmse(y1, y2):
    return np.sqrt(np.mean(np.square(y1 - y2)))

In [15]:
def to_minimize(x0, x, y, f):
    
    [p, w, m, h] = x0
    
    y2 = f(x, p, w, m, h)
    
    return rmse(y, y2)

In [16]:
x = np.linspace(-10., 10., 10001)

In [17]:
voigt(x, 0., 0.5, 0.5, 1.)

array([0.00045998, 0.00046349, 0.00046701, ..., 0.00046701, 0.00046349,
       0.00045998])

In [18]:
%%timeit

gls(x, 0., 1., 0.5, 1.)

120 µs ± 816 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [19]:
%%timeit

glp(x, 0., 1., 0.5, 1.)

117 µs ± 7.21 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [20]:
%%timeit

voigt(x, 0., 0.5, 0.5, 1.)

1.06 ms ± 56.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [91]:
c0 = np.array([0., 1., 1.])
dc = np.array([0., -1., 0.])

all_fit_params1 = []
all_fit_params2 = []

for compound, ppm, X, ws in zip(exp_compounds, ppms, Xs, all_ws):
    print(compound)
    for r in fit_ranges[compound]:
        print(f"Fitting {r} ppm")
        
        r0 = np.min(r)
        r1 = np.max(r)
        
        fdir = f"{fig_dir}{compound}_{r0:.2f}_{r1:.2f}_ppm/"
        if not os.path.exists(fdir):
            os.mkdir(fdir)
        
        fig = plt.figure(figsize=(4,3))
        ax = fig.add_subplot(1,1,1)
        for i, Xi in enumerate(X):
            ax.plot(ppm, Xi, color=c0 + i/(X.shape[0]-1) * dc, linewidth=1.)
        
        ax.set_xlim(20., -5.)
        ax.set_xlabel("Chemical shift [ppm]")
        ax.set_yticks([])
        fig.tight_layout()
        plt.savefig(f"{fdir}spectra.pdf")
        #plt.show()
        plt.close()
        
        fig = plt.figure(figsize=(4,3))
        ax = fig.add_subplot(1,1,1)
        for i, Xi in enumerate(X):
            ax.plot(ppm, Xi, color=c0 + i/(X.shape[0]-1) * dc, linewidth=1.)
        
        ax.set_xlim(r1, r0)
        ax.set_xlabel("Chemical shift [ppm]")
        ax.set_yticks([])
        fig.tight_layout()
        plt.savefig(f"{fdir}spectra_zoom.pdf")
        #plt.show()
        plt.close()
        
        fit_v = []
        fit_s = []
        fit_p = []
        
        fit_rmses_v = []
        fit_rmses_s = []
        fit_rmses_p = []
        
        inds = np.where(np.logical_and(ppm > r0, ppm < r1))
        
        for i, (wi, Xi) in tqdm.tqdm(enumerate(zip(ws, X)), total=len(ws)):
            
            p0 = get_maximum(ppm, Xi, r, method="interp")
            
            bounds[0] = (r0, r1)
            bounds_v[0] = (r0, r1)
            
            best_res_v = None
            best_res_s = None
            best_res_p = None
            for w0 in w0s:
                
                x0 = [p0, w0, w0, np.max(Xi[inds])]

                res_v = op.minimize(to_minimize, x0, args=(ppm[inds], Xi[inds], voigt), bounds=bounds_v,
                                    method="Nelder-Mead", options={"xatol": 1e-8, "fatol": 1e-8})

                x0 = [p0, w0, 0.5, np.max(Xi[inds])]
                
                res_s = op.minimize(to_minimize, x0, args=(ppm[inds], Xi[inds], gls), bounds=bounds,
                                    method="Nelder-Mead", options={"xatol": 1e-8, "fatol": 1e-8})
                
                x0 = [p0, w0, 0.5, np.max(Xi[inds])]
                
                res_p = op.minimize(to_minimize, x0, args=(ppm[inds], Xi[inds], glp), bounds=bounds,
                                    method="Nelder-Mead", options={"xatol": 1e-8, "fatol": 1e-8})
                
                if best_res_s is None:
                    best_res_v = res_v
                    best_res_s = res_s
                    best_res_p = res_p
                
                if res_v.fun < best_res_v.fun:
                    best_res_v = res_v
                if res_s.fun < best_res_s.fun:
                    best_res_s = res_s
                if res_p.fun < best_res_p.fun:
                    best_res_p = res_p
            
            #res_s = op.minimize(to_minimize, x0, args=(ppm[inds], Xi[inds], gls), bounds=bounds,
            #                    method="trust-constr", options={'maxiter': 100000})
            #res_p = op.minimize(to_minimize, x0, args=(ppm[inds], Xi[inds], glp), bounds=bounds,
            #                    method="trust-constr", options={'maxiter': 100000})
            
            fit_v.append(best_res_v.x)
            fit_s.append(best_res_s.x)
            fit_p.append(best_res_p.x)
            
            y_v = voigt(ppm[inds], res_v.x[0], res_v.x[1], res_v.x[2], res_v.x[3])
            y_s = gls(ppm[inds], res_s.x[0], res_s.x[1], res_s.x[2], res_s.x[3])
            y_p = glp(ppm[inds], res_s.x[0], res_s.x[1], res_s.x[2], res_s.x[3])
            
            fit_rmses_v.append(rmse(Xi[inds], y_v))
            fit_rmses_s.append(rmse(Xi[inds], y_s))
            fit_rmses_p.append(rmse(Xi[inds], y_p))
            
            fig = plt.figure(figsize=(3,3))
            ax = fig.add_subplot(1,1,1)
            
            ax.plot(ppm[inds], Xi[inds])
            ax.plot(ppm[inds], y_v)
            ax.plot(ppm[inds], y_s)
            ax.plot(ppm[inds], y_p)
            ax.set_yticks([])
            ax.set_ylim(0 - np.max(X[:, inds]) * 0.05, np.max(X[:, inds]) * 1.05)
            ax.set_xlim(r1, r0)
            ax.set_xlabel("Chemical shift [ppm]")
            
            ax.legend(["Data", "Voigt", "GLS", "GLP"])
            ax.plot([r0, r1], [0., 0.], "k", linewidth=0.5)
            fig.tight_layout()
            
            plt.savefig(f"{fdir}fit_{wi:.0f}_kHz.pdf")
            #plt.show()
            plt.close()
            
        fit_v = np.array(fit_v)
        fit_s = np.array(fit_s)
        fit_p = np.array(fit_p)
        
        w_inds = np.where(ws >= min_w_fit)
        
        these_fit_params1 = []
        these_fit_params2 = []
        
        for i, (fit, yl) in enumerate(zip(fit_v.T, ylabs_v)):
            
            fig = plt.figure(figsize=(4,3))
            ax = fig.add_subplot(1,1,1)

            P1, C1 = op.curve_fit(inverse_mas_fit, ws[w_inds], fit[w_inds])
            P2, C2 = op.curve_fit(inverse_mas2_fit, ws[w_inds], fit[w_inds])

            h0 = ax.scatter(ws / 1000, fit, s=5, c="C0")

            ax.set_ylabel(yl)
            ax.set_xlabel("MAS rate [kHz]")
            fig.tight_layout()
            plt.savefig(f"{fdir}Voigt_results_{yl}.pdf")
            
            h1 = ax.plot(ws / 1000, inverse_mas_fit(ws, P1[0], P1[1]), "C1")
            h2 = ax.plot(ws / 1000, inverse_mas2_fit(ws, P2[0], P2[1], P2[2]), "C2")

            ax.legend([h0, h1[0], h2[0]], ["Observed values", "1/w fit", "1/w + 1/w$^2$ fit"])

            ax.set_ylabel(yl)
            ax.set_xlabel("MAS rate [kHz]")
            fig.tight_layout()

            plt.savefig(f"{fdir}GLS_results_{yl}_fit.pdf")
            #plt.show()
            plt.close()
        
        for i, (fit, yl) in enumerate(zip(fit_s.T, ylabs_s)):
            
            fig = plt.figure(figsize=(4,3))
            ax = fig.add_subplot(1,1,1)
            
            if i == 1:
                fit_bounds = (0., np.inf)
            elif i == 2:
                fit_bounds = (0., np.inf)
            else:
                fit_bounds = (-np.inf, np.inf)

            P1, C1 = op.curve_fit(inverse_mas_fit, ws[w_inds], fit[w_inds], bounds=fit_bounds)
            P2, C2 = op.curve_fit(inverse_mas2_fit, ws[w_inds], fit[w_inds], bounds=fit_bounds)
            
            these_fit_params1.append(P1)
            these_fit_params2.append(P2)

            h0 = ax.scatter(ws / 1000, fit, s=5, c="C0")

            ax.set_ylabel(yl)
            ax.set_xlabel("MAS rate [kHz]")
            fig.tight_layout()
            plt.savefig(f"{fdir}GLS_results_{yl}.pdf")
            
            h1 = ax.plot(ws / 1000, inverse_mas_fit(ws, P1[0], P1[1]), "C1")
            h2 = ax.plot(ws / 1000, inverse_mas2_fit(ws, P2[0], P2[1], P2[2]), "C2")

            ax.legend([h0, h1[0], h2[0]], ["Observed values", "1/w fit", "1/w + 1/w$^2$ fit"])

            ax.set_ylabel(yl)
            ax.set_xlabel("MAS rate [kHz]")
            fig.tight_layout()

            plt.savefig(f"{fdir}GLS_results_{yl}_fit.pdf")
            #plt.show()
            plt.close()
            
        for i, (fit, yl) in enumerate(zip(fit_p.T, ylabs_p)):
            
            fig = plt.figure(figsize=(4,3))
            ax = fig.add_subplot(1,1,1)

            P1, C1 = op.curve_fit(inverse_mas_fit, ws[w_inds], fit[w_inds])
            P2, C2 = op.curve_fit(inverse_mas2_fit, ws[w_inds], fit[w_inds])

            h0 = ax.scatter(ws / 1000, fit, s=5, c="C0")
            
            ax.set_ylabel(yl)
            ax.set_xlabel("MAS rate [kHz]")
            fig.tight_layout()
            plt.savefig(f"{fdir}GLP_results_{yl}.pdf")
            
            h1 = ax.plot(ws / 1000, inverse_mas_fit(ws, P1[0], P1[1]), "C1")
            h2 = ax.plot(ws / 1000, inverse_mas2_fit(ws, P2[0], P2[1], P2[2]), "C2")

            ax.legend([h0, h1[0], h2[0]], ["Observed values", "1/w fit", "1/w + 1/w$^2$ fit"])

            ax.set_ylabel(yl)
            ax.set_xlabel("MAS rate [kHz]")
            fig.tight_layout()

            plt.savefig(f"{fdir}GLP_results_{yl}_fit.pdf")
            #plt.show()
            plt.close()
            
        fig = plt.figure(figsize=(4,3))
        ax = fig.add_subplot(1,1,1)
        ax.plot(ws / 1000., fit_rmses_s)
        ax.plot(ws / 1000., fit_rmses_p)
        ax.legend(["GLS", "GLP"])
        ax.set_ylabel("RMSE")
        ax.set_xlabel("MAS rate [kHz]")
        fig.tight_layout()
        plt.savefig(f"{fdir}rmses.pdf")
        #plt.show()
        plt.close()
        
        fig = plt.figure(figsize=(4,3))
        ax = fig.add_subplot(1,1,1)
        ax.plot(ws / 1000., fit_rmses_v)
        ax.plot(ws / 1000., fit_rmses_s)
        ax.plot(ws / 1000., fit_rmses_p)
        ax.legend(["Voigt", "GLS", "GLP"])
        ax.set_ylabel("RMSE")
        ax.set_xlabel("MAS rate [kHz]")
        fig.tight_layout()
        plt.savefig(f"{fdir}rmses_voigt.pdf")
        #plt.show()
        plt.close()
        
        all_fit_params1.append(these_fit_params1)
        all_fit_params2.append(these_fit_params2)

ampicillin
Fitting [9.5, 12.0] ppm


  0%|          | 0/41 [00:00<?, ?it/s]

//anaconda3/envs/torch/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
//anaconda3/envs/torch/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
//anaconda3/envs/torch/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


aspala
Fitting [12.0, 15.0] ppm


  0%|          | 0/38 [00:00<?, ?it/s]

...

Fitting [-2.0, 3.0] ppm


  0%|          | 0/36 [00:00<?, ?it/s]

mdma
Fitting [1.0, 3.8] ppm


  0%|          | 0/31 [00:00<?, ?it/s]

//anaconda3/envs/torch/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [92]:
all_fit_params1 = np.array(all_fit_params1)

In [93]:
all_fit_params2 = np.array(all_fit_params2)

In [94]:
labels = []
for compound in exp_compounds:
    for r in fit_ranges[compound]:
        labels.append(f"{compound}_{r[0]:.2f}_{r[1]:.2f}")

# Shift fitting

In [95]:
i = 0

pmin = []
pmax = []
pavg = []
pstd = []
for l, p in zip(labels, all_fit_params1[:, i, :]):
    print(f"{p[0]: .4e}\t{p[1]*800*2: .4e}\t{l}")

 9.9214e+00	 1.3800e+07	ampicillin_9.50_12.00
 1.2546e+01	 1.8287e+07	aspala_12.00_15.00
 8.0036e-01	 1.0747e+07	aspala_-2.00_1.50
 9.7867e+00	 1.9867e+07	flutamide_9.50_12.00
 1.6917e+01	 8.9226e+06	histidine_16.00_20.00
 1.2303e+01	 1.2975e+07	histidine_11.50_13.50
 5.0488e+00	 9.8563e+06	histidine_4.50_6.00
 9.3269e+00	 5.8853e+06	thymol_8.50_15.00
 1.2346e+01	 2.0974e+07	tyrosine_12.00_14.00
 9.9101e+00	 1.5912e+07	tyrosine_9.50_11.50
 2.4998e+00	-3.4254e+06	tyrosine_-2.00_3.00
 3.2677e+00	 1.3018e+07	mdma_1.00_3.80


In [96]:
i = 0

pmin = []
pmax = []
pavg = []
pstd = []
for l, p in zip(labels, all_fit_params2[:, i, :]):
    print(f"{p[0]: .4e}\t{p[1]*800*2: .4e}\t{p[2]*800*2: .4e}\t{l}")

 9.9214e+00	 1.3800e+07	 1.6000e+03	ampicillin_9.50_12.00
 1.2546e+01	 1.8287e+07	 1.6000e+03	aspala_12.00_15.00
 6.6730e-01	 3.8095e+07	-8.1487e+11	aspala_-2.00_1.50
 9.7867e+00	 1.9867e+07	 1.6000e+03	flutamide_9.50_12.00
 1.6917e+01	 8.9226e+06	 1.6000e+03	histidine_16.00_20.00
 1.2303e+01	 1.2975e+07	 1.6000e+03	histidine_11.50_13.50
 5.0488e+00	 9.8563e+06	 1.6000e+03	histidine_4.50_6.00
 9.3269e+00	 5.8853e+06	 1.6000e+03	thymol_8.50_15.00
 1.2346e+01	 2.0974e+07	 1.6000e+03	tyrosine_12.00_14.00
 9.9101e+00	 1.5912e+07	 1.6000e+03	tyrosine_9.50_11.50
 2.5539e+00	-1.4550e+07	 3.3146e+11	tyrosine_-2.00_3.00
 3.1345e+00	 4.0379e+07	-8.1524e+11	mdma_1.00_3.80


# Width fitting

In [97]:
i = 1

for l, p in zip(labels, all_fit_params1[:, i, :]):
    print(f"{p[0]*800*2: .4e}\t{p[1]*800*2: .4e}\t{l}")

 8.0205e+02	 3.5899e+07	ampicillin_9.50_12.00
 3.0235e+02	 1.5295e+07	aspala_12.00_15.00
 1.9191e+02	 3.3691e+07	aspala_-2.00_1.50
 6.3379e+02	 1.5688e+07	flutamide_9.50_12.00
 5.2329e+02	 1.8486e+07	histidine_16.00_20.00
 4.5001e+02	 1.9345e+07	histidine_11.50_13.50
 9.0755e+01	 6.0719e+07	histidine_4.50_6.00
 2.4626e+02	 1.2537e+08	thymol_8.50_15.00
 3.3379e+02	 1.8794e+07	tyrosine_12.00_14.00
 2.3851e+02	 2.9694e+07	tyrosine_9.50_11.50
 1.3271e-10	 8.4071e+07	tyrosine_-2.00_3.00
 8.5970e+01	 5.8535e+07	mdma_1.00_3.80


In [98]:
i = 1

for l, p in zip(labels, all_fit_params2[:, i, :]):
    print(f"{p[0]*800*2: .4e}\t{p[1]*800*2: .4e}\t{p[2]*800*2: .4e}\t{l}")

 8.0214e+02	 3.5888e+07	 3.3630e+08	ampicillin_9.50_12.00
 3.0235e+02	 1.5295e+07	 3.2305e+03	aspala_12.00_15.00
 1.9192e+02	 3.3690e+07	 1.4635e+03	aspala_-2.00_1.50
 6.3404e+02	 1.5672e+07	 1.3780e+03	flutamide_9.50_12.00
 5.2331e+02	 1.8483e+07	 7.6926e+07	histidine_16.00_20.00
 4.5002e+02	 1.9344e+07	 2.8378e+07	histidine_11.50_13.50
 9.0790e+01	 6.0715e+07	 1.3353e+08	histidine_4.50_6.00
 2.3061e+02	 1.2639e+08	 1.6002e+03	thymol_8.50_15.00
 3.3135e+02	 1.8953e+07	 4.4000e+02	tyrosine_12.00_14.00
 2.3860e+02	 2.9682e+07	 3.3941e+08	tyrosine_9.50_11.50
 5.0734e-08	 8.4068e+07	 2.0205e+08	tyrosine_-2.00_3.00
 8.6102e+01	 5.8518e+07	 5.0587e+08	mdma_1.00_3.80


# Mixing fitting

In [99]:
i = 2

for l, p in zip(labels, all_fit_params1[:, i, :]):
    print(f"{p[0]: .4e}\t{p[1]: .4e}\t{l}")

 3.5409e-13	 1.8395e+04	ampicillin_9.50_12.00
 3.8542e-02	 2.4862e+04	aspala_12.00_15.00
 8.0320e-02	 3.6960e+04	aspala_-2.00_1.50
 3.0299e-17	 1.1659e+04	flutamide_9.50_12.00
 8.2124e-02	 2.5879e+04	histidine_16.00_20.00
 3.0836e-16	 2.3746e+04	histidine_11.50_13.50
 1.5986e-01	 4.0321e+04	histidine_4.50_6.00
 1.2433e-12	 1.1911e+04	thymol_8.50_15.00
 9.6202e-02	 1.8681e+04	tyrosine_12.00_14.00
 4.3220e-02	 3.5019e+04	tyrosine_9.50_11.50
 6.5336e-01	 1.7816e+04	tyrosine_-2.00_3.00
 9.8602e-02	 3.2127e+04	mdma_1.00_3.80


In [100]:
i = 2

for l, p in zip(labels, all_fit_params2[:, i, :]):
    print(f"{p[0]: .4e}\t{p[1]: .4e}\t{p[2]: .4e}\t{l}")

 4.6513e-12	 1.8395e+04	 5.0095e+04	ampicillin_9.50_12.00
 3.8566e-02	 2.4859e+04	 9.4468e+04	aspala_12.00_15.00
 8.0320e-02	 3.6960e+04	 1.2924e-15	aspala_-2.00_1.50
 4.9129e-10	 1.1659e+04	 1.9408e+04	flutamide_9.50_12.00
 8.2124e-02	 2.5879e+04	 7.6383e+01	histidine_16.00_20.00
 1.4026e-11	 2.3745e+04	 6.7323e+04	histidine_11.50_13.50
 1.5496e-01	 4.0640e+04	 1.1349e+00	histidine_4.50_6.00
 1.6870e-11	 1.1912e+04	 3.1453e+04	thymol_8.50_15.00
 1.6514e-01	 1.4188e+04	 8.7125e-01	tyrosine_12.00_14.00
 4.3450e-02	 3.4990e+04	 8.7892e+05	tyrosine_9.50_11.50
 6.5336e-01	 1.7816e+04	 2.3807e-01	tyrosine_-2.00_3.00
 9.8602e-02	 3.2127e+04	 1.1143e+00	mdma_1.00_3.80


# Height fitting

In [101]:
i = 3

for l, p in zip(labels, all_fit_params1[:, i, :]):
    print(f"{p[0]: .4e}\t{p[1]: .4e}\t{l}")

 7.2696e-03	-1.6901e+02	ampicillin_9.50_12.00
 8.3133e-03	-1.4983e+02	aspala_12.00_15.00
 2.6093e-02	-7.3225e+02	aspala_-2.00_1.50
 4.7671e-03	-3.6747e+01	flutamide_9.50_12.00
 5.7850e-03	-9.8429e+01	histidine_16.00_20.00
 6.6487e-03	-1.1030e+02	histidine_11.50_13.50
 1.1696e-02	-3.7034e+02	histidine_4.50_6.00
 2.2219e-03	-5.3826e+01	thymol_8.50_15.00
 9.8353e-03	-2.3093e+02	tyrosine_12.00_14.00
 8.9383e-03	-2.2592e+02	tyrosine_9.50_11.50
 4.5600e-03	-1.3537e+02	tyrosine_-2.00_3.00
 1.8352e-02	-5.5146e+02	mdma_1.00_3.80


In [102]:
i = 3

for l, p in zip(labels, all_fit_params2[:, i, :]):
    print(f"{p[0]: .4e}\t{p[1]: .4e}\t{p[2]: .4e}\t{l}")

 9.4663e-03	-4.5121e+02	 8.4084e+06	ampicillin_9.50_12.00
 8.5195e-03	-1.7631e+02	 7.8926e+05	aspala_12.00_15.00
 3.4854e-02	-1.8578e+03	 3.3536e+07	aspala_-2.00_1.50
 4.3052e-03	 2.2584e+01	-1.7678e+06	flutamide_9.50_12.00
 6.3463e-03	-1.7053e+02	 2.1483e+06	histidine_16.00_20.00
 6.7806e-03	-1.2724e+02	 5.0486e+05	histidine_11.50_13.50
 1.4601e-02	-7.4356e+02	 1.1121e+07	histidine_4.50_6.00
 2.3059e-03	-6.4609e+01	 3.2130e+05	thymol_8.50_15.00
 1.1528e-02	-4.4843e+02	 6.4807e+06	tyrosine_12.00_14.00
 1.0672e-02	-4.4864e+02	 6.6361e+06	tyrosine_9.50_11.50
 5.6150e-03	-2.7089e+02	 4.0381e+06	tyrosine_-2.00_3.00
 2.7692e-02	-1.7514e+03	 3.5753e+07	mdma_1.00_3.80
